In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler(feature_range=(0,1))
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense,SimpleRNN

In [48]:
ticker = "ko"
months = 6
# aapl, msft, sber, mtss, moes, rosn, pfe, ko, intc, ma, ibm
df=pd.read_csv("data/" + ticker.upper() + "_5Y.csv")

In [49]:
MONTH_WORK_DAYS = 21
days_watching = 6*MONTH_WORK_DAYS
# if (months == 6):
#     days_watching = 9*MONTH_WORK_DAYS
    
predict_days = months*MONTH_WORK_DAYS
test_threshold = df.shape[0]

In [50]:
data = df.sort_index(ascending=True,axis=0)
df_new = pd.DataFrame(index=range(0,len(df)),columns=['Date','Close'])
for i in range(0,len(data)):
    df_new["Date"][i] = data['Date'][i]
    df_new["Close"][i] = data["Close"][i]
    
df_new.index = df_new.Date
df_new.drop("Date", axis=1, inplace=True)

final_dataset = df_new.values

train_data = final_dataset[0:test_threshold,:]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(final_dataset)

x_train_data,y_train_data = [],[]

for i in range(days_watching,len(train_data)-predict_days):
    x_train_data.append(scaled_data[i-days_watching:i,0])
    y_train_data.append(scaled_data[i+predict_days,0])
x_train_data,y_train_data = np.array(x_train_data),np.array(y_train_data)

x_train_data = np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

In [51]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50,return_sequences=False, input_shape=(x_train_data.shape[1],1)))
# lstm_model.add(LSTM(units=50,return_sequences=False, input_shape=(x_train_data.shape[1],1)))
# lstm_model.add(LSTM(units=128, return_sequences=True))
# lstm_model.add(LSTM(units=64, return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mse',optimizer='adam')
# lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=30,batch_size=40,verbose=2, shuffle=False)
lstm_model.save("compiled_models/" + ticker + "_" + str(months) + "m.h5")

Epoch 1/30
26/26 - 1s - loss: 0.0967
Epoch 2/30
26/26 - 1s - loss: 0.0382
Epoch 3/30
26/26 - 1s - loss: 0.0354
Epoch 4/30
26/26 - 1s - loss: 0.0338
Epoch 5/30
26/26 - 1s - loss: 0.0337
Epoch 6/30
26/26 - 1s - loss: 0.0334
Epoch 7/30
26/26 - 1s - loss: 0.0334
Epoch 8/30
26/26 - 1s - loss: 0.0333
Epoch 9/30
26/26 - 1s - loss: 0.0333
Epoch 10/30
26/26 - 1s - loss: 0.0332
Epoch 11/30
26/26 - 1s - loss: 0.0330
Epoch 12/30
26/26 - 1s - loss: 0.0329
Epoch 13/30
26/26 - 1s - loss: 0.0328
Epoch 14/30
26/26 - 1s - loss: 0.0327
Epoch 15/30
26/26 - 1s - loss: 0.0326
Epoch 16/30
26/26 - 1s - loss: 0.0325
Epoch 17/30
26/26 - 1s - loss: 0.0324
Epoch 18/30
26/26 - 1s - loss: 0.0323
Epoch 19/30
26/26 - 1s - loss: 0.0322
Epoch 20/30
26/26 - 1s - loss: 0.0321
Epoch 21/30
26/26 - 1s - loss: 0.0321
Epoch 22/30
26/26 - 1s - loss: 0.0320
Epoch 23/30
26/26 - 1s - loss: 0.0319
Epoch 24/30
26/26 - 1s - loss: 0.0318
Epoch 25/30
26/26 - 1s - loss: 0.0317
Epoch 26/30
26/26 - 1s - loss: 0.0316
Epoch 27/30
26/26 - 1